In [ ]:
import pandas as pd

# Load Excel file
file_path = "GIRS2.xlsx"
xl = pd.ExcelFile(file_path)

# Load necessary sheets
df_conso = xl.parse("Conso Data")
df_girs = xl.parse("GIRS")
df_tier = xl.parse("Tier_Data")
df_ttrl = xl.parse("TTRL_Data")
df_pdmr = xl.parse("PDMR_Data", header=None, names=["Resource Name"])

# Remove extra spaces & standardize case
def clean_dataframe(df):
    df.columns = df.columns.str.strip()
    return df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

df_conso = clean_dataframe(df_conso)
df_girs = clean_dataframe(df_girs)
df_tier = clean_dataframe(df_tier)
df_ttrl = clean_dataframe(df_ttrl)
df_pdmr = clean_dataframe(df_pdmr)

# Process unique PSIDs with Resource Name and Email
def process_psid_group(group):
    resource_names = [name.strip() for name in group["Resource Name"].dropna().unique()]
    email_ids = [email.strip() for email in group["Email"].dropna().unique()]
    psid = group["PSID"].iloc[0]
    
    # Remove resource names containing PSID unless it's the only one
    valid_names = [name for name in resource_names if str(psid) not in name]
    resource_name = max(valid_names, key=lambda x: len(x.split()), default=resource_names[0] if resource_names else "")
    
    # Validate email addresses
    valid_emails = [email for email in email_ids if "@" in email and not any(sym in email for sym in ["/", "#"])]
    email = "; ".join(valid_emails) if valid_emails else (email_ids[0] if email_ids else "")
    
    return pd.Series({"Resource Name": resource_name, "Email": email})

unique_psids = df_conso.groupby("PSID").apply(process_psid_group).reset_index()

# Capture invalid email records
invalid_emails = df_conso[df_conso["PSID"].isin(unique_psids[unique_psids["Email"].eq("")]["PSID"])]

# Merge GIRS Current Status
unique_psids = unique_psids.merge(df_girs[["PSID", "Role"]], on="PSID", how="left")
unique_psids.rename(columns={"Role": "GIRS Current Status"}, inplace=True)

# Merge Tier1 & TTRL status
unique_psids["Tier1"] = unique_psids["PSID"].isin(df_tier["PSID"]).map({True: "Yes", False: "No"})
unique_psids["TTRL"] = unique_psids["PSID"].isin(df_ttrl["PSID"]).map({True: "Yes", False: "No"})

# Merge PDMR status
unique_psids["PDMR"] = unique_psids["Resource Name"].isin(df_pdmr["Resource Name"]).map({True: "Yes", False: "No"})

# Extract unique team names from Conso Data Tab data column
df_conso["Tab data"] = df_conso["Tab data"].astype(str)  # Convert to string to avoid errors
unique_teams = df_conso["Tab data"].str.split(",").explode().str.strip().unique()

# Initialize team columns with N/A
for team in unique_teams:
    unique_psids[team] = "N/A"

# Update team status based on Conso Data
for _, row in df_conso.iterrows():
    psid = row["PSID"]
    status = row["Status"]
    teams = str(row["Tab data"]).split(",")  # Convert to string before splitting
    for team in teams:
        team = team.strip()
        unique_psids.loc[unique_psids["PSID"] == psid, team] = status

# Handle records with missing or non-numeric PSIDs
invalid_psid_records = df_conso[df_conso["PSID"].isna() | ~df_conso["PSID"].astype(str).str.isnumeric()]

# Save the final processed data
with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    unique_psids.to_excel(writer, sheet_name="Processed Data", index=False)
    invalid_emails.to_excel(writer, sheet_name="Invalid Emails", index=False)
    invalid_psid_records.to_excel(writer, sheet_name="Invalid PSIDs", index=False)


In [ ]:
import pandas as pd

# Load Excel file
file_path = "your_excel_file.xlsx"
xl = pd.ExcelFile(file_path)

# Load necessary sheets
df_conso = xl.parse("Conso Data")
df_girs = xl.parse("GIRS")
df_tier = xl.parse("Tier_Data")
df_ttrl = xl.parse("TTRL_Data")
df_pdmr = xl.parse("PDMR_Data")

# Remove extra spaces & standardize case
def clean_dataframe(df):
    df.columns = df.columns.str.strip()
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df_conso = clean_dataframe(df_conso)
df_girs = clean_dataframe(df_girs)
df_tier = clean_dataframe(df_tier)
df_ttrl = clean_dataframe(df_ttrl)
df_pdmr = clean_dataframe(df_pdmr)

# Extract unique PSIDs
unique_psids = df_conso[["PSID", "Resource Name", "Email"]].drop_duplicates()

# Merge GIRS Current Status
unique_psids = unique_psids.merge(df_girs[["PSID", "Role"]], on="PSID", how="left")
unique_psids.rename(columns={"Role": "GIRS Current Status"}, inplace=True)

# Merge Tier1 & TTRL status
unique_psids["Tier1"] = unique_psids["PSID"].isin(df_tier["PSID"]).map({True: "Yes", False: "No"})
unique_psids["TTRL"] = unique_psids["PSID"].isin(df_ttrl["PSID"]).map({True: "Yes", False: "No"})

# Merge PDMR status
unique_psids["PDMR"] = unique_psids["Resource Name"].isin(df_pdmr["Resource Name"]).map({True: "Yes", False: "No"})

# Extract unique team names from Conso Data Tab Data column
unique_teams = df_conso["Tab Data"].str.split(",").explode().str.strip().unique()

# Initialize team columns with N/A
for team in unique_teams:
    unique_psids[team] = "N/A"

# Update team status based on Conso Data
for _, row in df_conso.iterrows():
    psid = row["PSID"]
    status = row["Status"]
    teams = row["Tab Data"].split(",")
    for team in teams:
        team = team.strip()
        unique_psids.loc[unique_psids["PSID"] == psid, team] = status

# Save the final processed data
with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace') as writer:
    unique_psids.to_excel(writer, sheet_name="Processed Data", index=False)


In [ ]:
import pandas as pd

# Load Excel file
file_path = "your_file.xlsx"
xls = pd.ExcelFile(file_path)

# Read required sheets
df_conso = pd.read_excel(xls, sheet_name='Conso Data')
df_girs = pd.read_excel(xls, sheet_name="GIRS Extract_20 Mar'25")
df_tier1 = pd.read_excel(xls, sheet_name="TIER1_5 Mar'25")
df_ttrl = pd.read_excel(xls, sheet_name="TTRL_3 Mar'25")
df_pdmr = pd.read_excel(xls, sheet_name="PDMR_1Jan'25", header=None, names=['PDMR'])

# Data Cleaning & Standardization
df_conso['PSID'] = df_conso['PSID'].astype(str).str.lstrip('0')
df_conso['Resource name'] = df_conso['Resource name'].str.title()
df_pdmr['PDMR'] = df_pdmr['PDMR'].str.lower()

df_girs['Employee ID'] = df_girs['Employee ID'].astype(str).str.lstrip('0')
df_girs['Insider Name'] = df_girs['Insider Name'].str.title()

df_tier1['Employee ID'] = df_tier1['Employee ID'].astype(str).str.lstrip('0')
df_tier1['Insider Name'] = df_tier1['Insider Name'].str.title()

df_ttrl['Employee ID'] = df_ttrl['Employee ID'].astype(str).str.lstrip('0')
df_ttrl['Insider Name'] = df_ttrl['Insider Name'].str.title()

# Merge Conso Data with GIRS Extract
df_merged = df_conso.merge(df_girs, left_on='PSID', right_on='Employee ID', how='left')

# Identify status updates
final_records = []
for _, row in df_merged.iterrows():
    psid = row['PSID']
    resource_name = row['Resource name']
    email = row['Email']
    original_status = row['Role'] if pd.notna(row['Role']) else 'Not in GIRS'
    updated_status = row['Status']
    comments = []
    
    if row['TIER1'] == 'Yes' or row['TTRL'] == 'Yes' or resource_name.lower() in df_pdmr['PDMR'].values:
        comments.append("Employee in TIER1/TTRL/PDMR")
    
    if original_status == 'Non-Core' and updated_status == 'DEL':
        updated_status = 'Inactive Insider'
        comments.append("Non-Core user marked for deletion")
    
    if original_status == 'Inactive Insider' and updated_status == 'ADD':
        updated_status = 'Non-Core'
        comments.append("Inactive Insider reactivated as Non-Core")
    
    final_records.append([psid, resource_name, email, original_status, updated_status, ', '.join(comments)])

# Convert to DataFrame
final_df = pd.DataFrame(final_records, columns=['PSID', 'Resource name', 'Email', 'Original Status', 'Updated Status', 'Comments'])

# Export to Excel
final_df.to_excel("Processed_GIRS_Data.xlsx", index=False)


In [ ]:
import pandas as pd

def process_girs_data(file_path):
    # Load required sheets
    conso_data = pd.read_excel(file_path, sheet_name='Conso Data')
    girs_data = pd.read_excel(file_path, sheet_name="GIRS Extract_20 Mar'25")
    tier1_data = pd.read_excel(file_path, sheet_name='TIER1_5 Mar'25')
    ttrl_data = pd.read_excel(file_path, sheet_name='TTRL_3 Mar'25')
    pdmr_data = pd.read_excel(file_path, sheet_name='PDMR_1Jan'25', header=None)  # PDMR has one column
    
    # Standardizing column names
    conso_data.rename(columns={'Resource name': 'Insider Name'}, inplace=True)
    girs_data.rename(columns={'Employee ID': 'PSID', 'Employee Email': 'Email'}, inplace=True)
    tier1_data.rename(columns={'Employee ID': 'PSID', 'Employee Email': 'Email'}, inplace=True)
    ttrl_data.rename(columns={'Employee ID': 'PSID', 'Employee Email': 'Email'}, inplace=True)
    
    # Convert 'PDMR' column to lowercase for matching
    pdmr_data[0] = pdmr_data[0].str.lower()
    
    # Remove leading zeros from PSID
    conso_data['PSID'] = conso_data['PSID'].astype(str).str.lstrip('0')
    girs_data['PSID'] = girs_data['PSID'].astype(str).str.lstrip('0')
    tier1_data['PSID'] = tier1_data['PSID'].astype(str).str.lstrip('0')
    ttrl_data['PSID'] = ttrl_data['PSID'].astype(str).str.lstrip('0')
    
    # Capitalize Insider Names
    conso_data['Insider Name'] = conso_data['Insider Name'].str.upper()
    girs_data['Insider Name'] = girs_data['Insider Name'].str.upper()
    tier1_data['Insider Name'] = tier1_data['Insider Name'].str.upper()
    ttrl_data['Insider Name'] = ttrl_data['Insider Name'].str.upper()
    
    # Merge GIRS Current Status into Conso Data
    merged_data = conso_data.merge(girs_data[['PSID', 'Insider Name', 'Email', 'Role']], 
                                   on=['PSID', 'Insider Name'], how='left')
    merged_data.rename(columns={'Role': 'Original Status in GIRS'}, inplace=True)
    
    # Identify if PSID is present in Tier1, TTRL
    merged_data['Tier1 Flag'] = merged_data['PSID'].isin(tier1_data['PSID']).map({True: 'Yes', False: 'No'})
    merged_data['TTRL Flag'] = merged_data['PSID'].isin(ttrl_data['PSID']).map({True: 'Yes', False: 'No'})
    
    # Identify if Insider Name matches with PDMR data
    merged_data['PDMR Flag'] = merged_data['Insider Name'].str.lower().isin(pdmr_data[0]).map({True: 'Yes', False: 'No'})
    
    # Determine Updated Status Request based on rules
    def determine_updated_status(row):
        if row['Status'] == 'ADD':
            return 'Non-Core' if row['Original Status in GIRS'] == 'Inactive Insider' else 'ADD'
        elif row['Status'] == 'DEL':
            return 'Inactive Insider' if row['Original Status in GIRS'] == 'Non-Core' else 'DEL'
        elif row['Status'] == 'Cur':
            return row['Original Status in GIRS']  # Keep as is
        return row['Status']
    
    merged_data['Updated Status Request'] = merged_data.apply(determine_updated_status, axis=1)
    
    # Identify Request Type (New/Updated Request)
    merged_data['Request Type'] = merged_data['Original Status in GIRS'].apply(
        lambda x: 'New Request' if pd.isna(x) else 'Updated Request'
    )
    
    # Generate Comments Column
    def generate_comment(row):
        comments = []
        if row['Status'] == 'ADD' and row['Updated Status Request'] == 'ADD':
            comments.append('Fresh Addition')
        if row['Status'] == 'DEL' and row['Updated Status Request'] == 'Inactive Insider':
            comments.append('Converted to Inactive Insider')
        if row['Status'] == 'ADD' and row['Updated Status Request'] == 'Non-Core':
            comments.append('Reactivated to Non-Core')
        if row['Tier1 Flag'] == 'Yes':
            comments.append('Present in Tier1')
        if row['TTRL Flag'] == 'Yes':
            comments.append('Present in TTRL')
        if row['PDMR Flag'] == 'Yes':
            comments.append('Listed in PDMR')
        return '; '.join(comments)
    
    merged_data['Comment'] = merged_data.apply(generate_comment, axis=1)
    
    # Select final required columns
    final_columns = ['PSID', 'Insider Name', 'Email', 'Original Status in GIRS',
                     'Updated Status Request', 'Request Type', 'Comment']
    final_data = merged_data[final_columns]
    
    # Save output to a new Excel file
    output_file = 'Final_Consolidated_Data.xlsx'
    final_data.to_excel(output_file, index=False)
    print(f'Final consolidated data saved as {output_file}')
    
    return final_data

# Example usage:
file_path = 'GIRS_Processing.xlsx'  # Update this with your actual file path
final_output = process_girs_data(file_path)
